## The simplest pattern in multiprocessing is creating a new process using the Process class.

In [ ]:
import multiprocessing
import os


def worker():
    print(f"Worker process started: {os.getpid()}")


if __name__ == "__main__":
    p = multiprocessing.Process(target=worker)
    p.start()  # Start the process
    p.join()  # Wait for the process to finish

In [ ]:
import multiprocessing
import os
import time


def worker_a():
    print(f"Worker A started: {os.getpid()}")
    time.sleep(2)  # Simulate work
    print(f"Worker A finished: {os.getpid()}")


def worker_b():
    print(f"Worker B started: {os.getpid()}")
    time.sleep(3)  # Simulate work
    print(f"Worker B finished: {os.getpid()}")


def worker_c():
    print(f"Worker C started: {os.getpid()}")
    time.sleep(1)  # Simulate work
    print(f"Worker C finished: {os.getpid()}")


if __name__ == "__main__":
    # Create process instances for each worker function
    processes = [
        multiprocessing.Process(target=worker_a),
        multiprocessing.Process(target=worker_b),
        multiprocessing.Process(target=worker_c),
    ]

    # Start all processes
    for p in processes:
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

## 2. You can pass arguments to the target function when creating a new process.

In [ ]:
import multiprocessing


def worker(num):
    print(f"Worker {num} started")


if __name__ == "__main__":
    processes = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()  # Wait for all processes to finish

## 3. A Pool is used to manage a pool of worker processes that can perform tasks in parallel. This is especially useful for distributing a large number of tasks across multiple processes.

In [ ]:
import multiprocessing


def square(x):
    return x * x


if __name__ == "__main__":
    with multiprocessing.Pool(4) as pool:  # upto 4 task can run in paraleel
        result = pool.map(square, range(10))  # Run the square function on each input
    print(result)

In [ ]:
import multiprocessing
import time


def square(x):
    time.sleep(0.5)
    print(x * x)
    return x * x


if __name__ == "__main__":
    with multiprocessing.Pool(4) as pool:
        result = pool.map(square, range(10))  # Run the square function on each input
    print(result)

## 4. Inter-process Communication with multiprocessing.Queue multiprocessing.Queue is used for passing messages between processes safely.

In [ ]:
import multiprocessing


def producer(queue):
    for i in range(5):
        queue.put(i)  # Put items in the queue
    queue.put(None)  # Sentinel value to signal completion


def consumer(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        print(f"Consumed {item}")


if __name__ == "__main__":
    queue = multiprocessing.Queue()
    p1 = multiprocessing.Process(target=producer, args=(queue,))
    p2 = multiprocessing.Process(target=consumer, args=(queue,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

## 5. Inter-process Communication with multiprocessing.Pipe


In [ ]:
import multiprocessing


def worker(conn):
    conn.send("Hello from worker!")
    conn.close()


if __name__ == "__main__":
    parent_conn, child_conn = multiprocessing.Pipe()
    p = multiprocessing.Process(target=worker, args=(child_conn,))

    p.start()
    print(parent_conn.recv())  # Receive the message from the worker
    p.join()

In [ ]:
import multiprocessing


def worker(conn):
    message = "Hello from worker!"
    conn.send(message)  # Send message to parent
    conn.close()  # Close the connection


if __name__ == "__main__":
    parent_conn, child_conn = multiprocessing.Pipe()  # Create a pipe
    p = multiprocessing.Process(target=worker, args=(child_conn,))  # Initialize Process

    p.start()  # Start the worker process
    received_msg = parent_conn.recv()  # Receive message from worker
    print(
        f"Parent received: {received_msg}"
    )  # Output: Parent received: Hello from worker!
    p.join()  # Wait for the worker to finish

In [ ]:
import multiprocessing


def worker(conn):
    # Receive a message from the parent
    parent_msg = conn.recv()
    print(f"Worker received: {parent_msg}")

    # Send a reply back to the parent
    reply = "Hello from worker!"
    conn.send(reply)
    conn.close()


if __name__ == "__main__":
    parent_conn, child_conn = multiprocessing.Pipe()

    p = multiprocessing.Process(target=worker, args=(child_conn,))
    p.start()

    # Parent sends a message to the worker
    parent_conn.send("Hello from parent!")

    # Parent receives a reply from the worker
    reply = parent_conn.recv()
    print(f"Parent received: {reply}")

    p.join()

## 6. multiprocessing.Lock ensures that only one process can access a critical section at a time, avoiding race conditions.

In [ ]:
import multiprocessing


def worker(lock, shared_value):
    with lock:
        for _ in range(1000):
            shared_value.value += 1


if __name__ == "__main__":
    lock = multiprocessing.Lock()
    shared_value = multiprocessing.Value("i", 0)

    processes = [
        multiprocessing.Process(target=worker, args=(lock, shared_value))
        for _ in range(4)
    ]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    print(f"Shared value: {shared_value.value}")

## 7. Using multiprocessing.Semaphore for Controlling Access

In [ ]:
import multiprocessing
import time


def worker(semaphore, name):
    with semaphore:
        print(f"{name} is working...")
        time.sleep(1)
        print(f"{name} finished work.")


if __name__ == "__main__":
    semaphore = multiprocessing.Semaphore(2)  # Only allow 2 workers at a time

    processes = [
        multiprocessing.Process(target=worker, args=(semaphore, f"Worker {i}"))
        for i in range(5)
    ]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

## 8. Using multiprocessing.Event for Signaling Between Processes

In [ ]:
import multiprocessing
import time


def wait_for_event(event):
    print("Waiting for event...")
    event.wait()  # Wait until the event is set
    print("Event received, continuing work...")


def trigger_event(event):
    time.sleep(2)
    print("Setting event")
    event.set()


if __name__ == "__main__":
    event = multiprocessing.Event()

    p1 = multiprocessing.Process(target=wait_for_event, args=(event,))
    p2 = multiprocessing.Process(target=trigger_event, args=(event,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

## 9. multiprocessing.Manager allows sharing of complex data types (like lists, dictionaries) between processes.

In [ ]:
import multiprocessing


def worker(shared_list, shared_dict):
    shared_list.append(1)
    shared_dict["count"] = shared_dict.get("count", 0) + 1


if __name__ == "__main__":
    manager = multiprocessing.Manager()
    shared_list = manager.list()  # Shared list
    shared_dict = manager.dict()  # Shared dictionary

    processes = [
        multiprocessing.Process(target=worker, args=(shared_list, shared_dict))
        for _ in range(5)
    ]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    print(f"Shared list: {list(shared_list)}")
    print(f"Shared dict: {dict(shared_dict)}")